# Instalacion

In [1]:
! pip install sqlalchemy

# Configurando la base de datos

Vamos a crear una base de datos simple para gestionar una biblioteca. Tendrá dos tablas principales: Libro y Autor.

* Cada libro tiene un título, una fecha de publicación y un autor asociado.
* Cada autor tiene un nombre y una nacionalidad.
* Queremos poder consultar los libros por autor y realizar algunas consultas simples.

In [1]:
# import sqlalchemy

from sqlalchemy import create_engine, Column, Integer, String, Date, ForeignKey
from sqlalchemy.orm import declarative_base, relationship, sessionmaker
from datetime import date

In [2]:
# Configuración de la conexión y el motor de la base de datos, en este caso usaremos sqlite para no tener que configurar la base de datos en el sistema
engine = create_engine('sqlite:///biblioteca.db', echo=True)
Base = declarative_base() # Crear una clase base desde la cual todas las clases de modelo de base de datos heredan.

In [3]:
# Definimos la clase `Autor`, que representa la tabla "autores" en la base de datos.
# Hereda de `Base`, lo que indica a SQLAlchemy que esta clase es un modelo de tabla.
class Autor(Base):
    # Nombre de la tabla en la base de datos
    __tablename__ = 'autores'
    
    # Columna 'id' como clave primaria: cada autor tendrá un ID único para identificarlo en la tabla
    id = Column(Integer, primary_key=True)
    
    # Columna 'nombre' para almacenar el nombre del autor. 
    # 'nullable=False' significa que este campo no puede estar vacío.
    nombre = Column(String, nullable=False)
    
    # Columna 'nacionalidad' para almacenar la nacionalidad del autor.
    # También se marca como no nula para asegurar que cada autor tenga nacionalidad registrada.
    nacionalidad = Column(String, nullable=False)
    
    # Definición de relación con la tabla 'libros'.
    # Esto establece una relación uno-a-muchos entre 'Autor' y 'Libro',
    # donde un autor puede tener varios libros.
    # `back_populates="autor"` conecta esta relación con la relación en la clase `Libro`.
    libros = relationship("Libro", back_populates="autor")
    
# Definición de la tabla Libro
class Libro(Base):
    __tablename__ = 'libros'
    
    id = Column(Integer, primary_key=True)
    titulo = Column(String, nullable=False)
    fecha_publicacion = Column(Date, nullable=False)
    
    # Columna 'autor_id' que actúa como clave foránea, conectando este libro con un autor en la tabla `autores`.
    # La clave foránea `ForeignKey('autores.id')` establece que `autor_id` debe coincidir con un `id` en la tabla `autores`.
    autor_id = Column(Integer, ForeignKey('autores.id'))
    
    # Relación inversa con la tabla `Autor`.
    # `back_populates="libros"` conecta esta relación con la relación en la clase `Autor`.
    # Esto permite acceder al autor de un libro usando `libro.autor`, y acceder a los libros de un autor usando `autor.libros`.
    autor = relationship("Autor", back_populates="libros")

In [4]:
# Crear las tablas en la base de datos
# ---------------------------------------------------
# `Base.metadata.create_all(engine)` es una llamada que utiliza la metadata de todos los modelos de tabla definidos 
# (es decir, todos los modelos que heredan de `Base`) para crear esas tablas en la base de datos.
# La metadata es una colección de definiciones de esquema de tablas, incluyendo nombres de tablas, columnas, claves 
# primarias y relaciones.
# Al ejecutar `create_all`, SQLAlchemy revisa las tablas de la base de datos que no existen y las crea de acuerdo 
# a las especificaciones de nuestros modelos.
# Nota: Si las tablas ya existen, `create_all` no las reemplazará, lo que evita la pérdida de datos.
Base.metadata.create_all(engine)

# Configuración de la sesión
# ---------------------------------------------------
# La sesión es el punto de interacción principal entre el código Python y la base de datos.
# En SQLAlchemy, una "sesión" es un objeto que gestiona operaciones de consulta y transacción.
# Aquí configuramos una sesión para conectarnos a la base de datos y realizar consultas, inserciones, 
# actualizaciones y eliminaciones en las tablas.

# Primero, creamos una clase de fábrica de sesiones (`sessionmaker`) que está enlazada (bind) al `engine`.
# `engine` es el objeto que define la conexión a la base de datos, configurando así `sessionmaker` para crear 
# sesiones que interactúan con esa base de datos específica.
Session = sessionmaker(bind=engine)

2024-11-14 11:09:57,669 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-11-14 11:09:57,670 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("autores")
2024-11-14 11:09:57,671 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-11-14 11:09:57,673 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("autores")
2024-11-14 11:09:57,674 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-11-14 11:09:57,675 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("libros")
2024-11-14 11:09:57,676 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-11-14 11:09:57,676 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("libros")
2024-11-14 11:09:57,677 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-11-14 11:09:57,678 INFO sqlalchemy.engine.Engine 
CREATE TABLE autores (
	id INTEGER NOT NULL, 
	nombre VARCHAR NOT NULL, 
	nacionalidad VARCHAR NOT NULL, 
	PRIMARY KEY (id)
)


2024-11-14 11:09:57,678 INFO sqlalchemy.engine.Engine [no key 0.00066s] ()
2024-11-14 11:09:57,684 INFO sqlalchemy.engine.Engine

In [5]:
# `Session()` crea una instancia de sesión que se usará para interactuar con la base de datos.
# Esta instancia de sesión nos permite agregar, eliminar y consultar registros en la base de datos.
# Las operaciones de la sesión son transaccionales: hasta que llamemos a `commit()`, los cambios no se 
# guardan permanentemente en la base de datos. Esto proporciona control sobre las transacciones, permitiendo 
# confirmar (`commit`) o deshacer (`rollback`) las operaciones según sea necesario.
session = Session()

# Insertar datos en la base de datos

In [6]:
# Crear instancias de autores
autor1 = Autor(nombre="Gabriel García Márquez", nacionalidad="Colombiana")
autor2 = Autor(nombre="Isabel Allende", nacionalidad="Chilena")

# Crear instancias de libros asociados a los autores
libro1 = Libro(titulo="Cien años de soledad", fecha_publicacion=date(1967, 5, 5), autor=autor1)
libro2 = Libro(titulo="El amor en los tiempos del cólera", fecha_publicacion=date(1985, 3, 6), autor=autor1)
libro3 = Libro(titulo="La casa de los espíritus", fecha_publicacion=date(1982, 10, 15), autor=autor2)

# Agregar y confirmar los cambios en la sesión
session.add_all([autor1, autor2, libro1, libro2, libro3])
session.commit()

2024-11-14 11:10:07,930 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-11-14 11:10:07,933 INFO sqlalchemy.engine.Engine INSERT INTO autores (nombre, nacionalidad) VALUES (?, ?) RETURNING id
2024-11-14 11:10:07,934 INFO sqlalchemy.engine.Engine [generated in 0.00008s (insertmanyvalues) 1/2 (ordered; batch not supported)] ('Gabriel García Márquez', 'Colombiana')
2024-11-14 11:10:07,935 INFO sqlalchemy.engine.Engine INSERT INTO autores (nombre, nacionalidad) VALUES (?, ?) RETURNING id
2024-11-14 11:10:07,936 INFO sqlalchemy.engine.Engine [insertmanyvalues 2/2 (ordered; batch not supported)] ('Isabel Allende', 'Chilena')
2024-11-14 11:10:07,938 INFO sqlalchemy.engine.Engine INSERT INTO libros (titulo, fecha_publicacion, autor_id) VALUES (?, ?, ?) RETURNING id
2024-11-14 11:10:07,939 INFO sqlalchemy.engine.Engine [generated in 0.00016s (insertmanyvalues) 1/3 (ordered; batch not supported)] ('Cien años de soledad', '1967-05-05', 1)
2024-11-14 11:10:07,941 INFO sqlalchemy.engine.Engine I

# Consultas

In [7]:
libros = session.query(Libro).all()
for libro in libros:
    print(f"{libro.titulo}, escrito por {libro.autor.nombre}")

2024-11-14 11:10:11,448 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-11-14 11:10:11,452 INFO sqlalchemy.engine.Engine SELECT libros.id AS libros_id, libros.titulo AS libros_titulo, libros.fecha_publicacion AS libros_fecha_publicacion, libros.autor_id AS libros_autor_id 
FROM libros
2024-11-14 11:10:11,453 INFO sqlalchemy.engine.Engine [generated in 0.00083s] ()
2024-11-14 11:10:11,456 INFO sqlalchemy.engine.Engine SELECT autores.id AS autores_id, autores.nombre AS autores_nombre, autores.nacionalidad AS autores_nacionalidad 
FROM autores 
WHERE autores.id = ?
2024-11-14 11:10:11,458 INFO sqlalchemy.engine.Engine [generated in 0.00191s] (1,)
Cien años de soledad, escrito por Gabriel García Márquez
El amor en los tiempos del cólera, escrito por Gabriel García Márquez
2024-11-14 11:10:11,459 INFO sqlalchemy.engine.Engine SELECT autores.id AS autores_id, autores.nombre AS autores_nombre, autores.nacionalidad AS autores_nacionalidad 
FROM autores 
WHERE autores.id = ?
2024-11-14 11:1

In [10]:
# Consultas con filtros
autor = session.query(Autor).filter_by(nombre="Gabriel García Márquez").first()
if autor:
    for libro in autor.libros:
        print(libro.titulo)

2024-11-14 11:12:43,044 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-11-14 11:12:43,046 INFO sqlalchemy.engine.Engine SELECT autores.id AS autores_id, autores.nombre AS autores_nombre, autores.nacionalidad AS autores_nacionalidad 
FROM autores 
WHERE autores.nombre = ?
 LIMIT ? OFFSET ?
2024-11-14 11:12:43,047 INFO sqlalchemy.engine.Engine [cached since 145.3s ago] ('Gabriel García Márquez', 1, 0)
2024-11-14 11:12:43,051 INFO sqlalchemy.engine.Engine SELECT libros.id AS libros_id, libros.titulo AS libros_titulo, libros.fecha_publicacion AS libros_fecha_publicacion, libros.autor_id AS libros_autor_id 
FROM libros 
WHERE ? = libros.autor_id
2024-11-14 11:12:43,053 INFO sqlalchemy.engine.Engine [cached since 145.3s ago] (1,)
Cien años de soledad (Edición revisada)
El amor en los tiempos del cólera


In [9]:
# Actualizacion
libro = session.query(Libro).filter_by(titulo="Cien años de soledad").first()
if libro:
    libro.titulo = "Cien años de soledad (Edición revisada)"
    session.commit()

2024-11-14 11:12:35,650 INFO sqlalchemy.engine.Engine SELECT libros.id AS libros_id, libros.titulo AS libros_titulo, libros.fecha_publicacion AS libros_fecha_publicacion, libros.autor_id AS libros_autor_id 
FROM libros 
WHERE libros.titulo = ?
 LIMIT ? OFFSET ?
2024-11-14 11:12:35,652 INFO sqlalchemy.engine.Engine [generated in 0.00171s] ('Cien años de soledad', 1, 0)
2024-11-14 11:12:35,656 INFO sqlalchemy.engine.Engine UPDATE libros SET titulo=? WHERE libros.id = ?
2024-11-14 11:12:35,656 INFO sqlalchemy.engine.Engine [generated in 0.00080s] ('Cien años de soledad (Edición revisada)', 1)
2024-11-14 11:12:35,658 INFO sqlalchemy.engine.Engine COMMIT


In [ ]:
# Eliminar registro

libro = session.query(Libro).filter_by(titulo="La casa de los espíritus").first()
if libro:
    session.delete(libro)
    session.commit()

# Transacción

En SQLAlchemy, una transacción agrupa varias operaciones de base de datos, de modo que todas se confirmen o ninguna lo haga. Esto es útil si queremos asegurar que varias operaciones se realicen correctamente o se reviertan en caso de error.

In [ ]:
from sqlalchemy.exc import IntegrityError
session = Session()
# Iniciamos la transacción
try:
    # Creamos un nuevo autor
    nuevo_autor = Autor(nombre="Gabriel García Márquez", nacionalidad="Colombiano")
    
    # Creamos un nuevo libro asociado a este autor
    nuevo_libro = Libro(titulo="Crónica de una muerte anunciada", fecha_publicacion="1967-05-30", autor=nuevo_autor) # date(1981, 3, 6)
    
    # Añadimos el autor y el libro a la sesión
    session.add(nuevo_autor)
    session.add(nuevo_libro)
    
    # Confirmamos la transacción
    session.commit()
    print("Transacción completada con éxito.")
except IntegrityError as e:
    # En caso de error, revertimos todas las operaciones realizadas en esta transacción
    session.rollback()
    print(f"Error en la transacción: {e}")


2024-11-14 11:08:53,334 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-11-14 11:08:53,336 INFO sqlalchemy.engine.Engine INSERT INTO autores (nombre, nacionalidad) VALUES (?, ?)
2024-11-14 11:08:53,336 INFO sqlalchemy.engine.Engine [cached since 303.9s ago] ('Gabriel García Márquez', 'Colombiano')
2024-11-14 11:08:53,337 INFO sqlalchemy.engine.Engine INSERT INTO libros (titulo, fecha_publicacion, autor_id) VALUES (?, ?, ?)
2024-11-14 11:08:53,338 INFO sqlalchemy.engine.Engine [cached since 303.9s ago] ('Crónica de una muerte anunciada', '1981-03-06', 3)
2024-11-14 11:08:53,339 INFO sqlalchemy.engine.Engine COMMIT
Transacción completada con éxito.


In [ ]:
# Consultas con filtros
autor = session.query(Autor).filter_by(nombre="Gabriel García Márquez").first()
if autor:
    for libro in autor.libros:
        print(libro.titulo)

2024-11-14 11:08:56,798 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-11-14 11:08:56,799 INFO sqlalchemy.engine.Engine SELECT autores.id AS autores_id, autores.nombre AS autores_nombre, autores.nacionalidad AS autores_nacionalidad 
FROM autores 
WHERE autores.nombre = ?
 LIMIT ? OFFSET ?
2024-11-14 11:08:56,800 INFO sqlalchemy.engine.Engine [cached since 1886s ago] ('Gabriel García Márquez', 1, 0)
2024-11-14 11:08:56,802 INFO sqlalchemy.engine.Engine SELECT libros.id AS libros_id, libros.titulo AS libros_titulo, libros.fecha_publicacion AS libros_fecha_publicacion, libros.autor_id AS libros_autor_id 
FROM libros 
WHERE ? = libros.autor_id
2024-11-14 11:08:56,802 INFO sqlalchemy.engine.Engine [cached since 1886s ago] (1,)
Cien años de soledad
El amor en los tiempos del cólera


# Consulta Avanzada

Supongamos que queremos realizar una consulta avanzada que nos proporcione los autores y el número de libros que cada uno tiene en la base de datos. Utilizaremos una combinación de agregación y filtrado.

In [ ]:
from sqlalchemy import func

# Consulta avanzada para contar libros por autor
consulta = (
    session.query(Autor.nombre, func.count(Libro.id).label("num_libros"))
    .join(Libro, Autor.id == Libro.autor_id)  # Unimos Autor y Libro a través de autor_id
    .group_by(Autor.id)  # Agrupamos por el autor para contar sus libros
    .having(func.count(Libro.id) > 1)  # Filtramos para autores con más de un libro
    .all()
)

# Mostramos los resultados
for autor, num_libros in consulta:
    print(f"Autor: {autor}, Libros publicados: {num_libros}")